To use this Notebook you have to import the StaffPlanning example (New Decision Optimization Model/From File)


### Client

Create a DODS client to connect to initial scenario.

In [1]:
from dd_scenario import *

client = Client()
decision = client.get_model_builder(name="StaffPlanning")
scenario = decision.get_scenario(name="Scenario 1")


### Global parameters

The number of days and number of periods per day:

In [2]:
N_DAYS = 2
N_PERIODS_PER_DAY = 24*4
N_PERIODS = N_DAYS * N_PERIODS_PER_DAY

### Random generator

A method to generate the random demand for the given number of days and periods. 

In [3]:
import random
import numpy as np
import pandas as pd

def random_demand( b_size ):
    rs = []
    for d in range(N_DAYS):
        # Morning
        p1 = random.uniform(0.2, 0.4)
        s1 = int(random.uniform(b_size*0.5, b_size*1.5))
        rs.append(np.random.binomial(n=N_PERIODS_PER_DAY, p=p1, size=s1) + d*N_PERIODS_PER_DAY)
        # Afternoon
        p2 = random.uniform(0.6, 0.8)
        s2 = int(random.uniform(b_size*0.5, b_size*1.5))
        rs.append(np.random.binomial(n=N_PERIODS_PER_DAY, p=p2, size=s2) + d*N_PERIODS_PER_DAY)
        # Rest of day
        s3 = int(random.uniform(b_size*0.4, b_size*0.7))
        e = np.array([ random.randint(int(d*N_PERIODS_PER_DAY + 0.2*N_PERIODS_PER_DAY), int(d*N_PERIODS_PER_DAY + 0.8*N_PERIODS_PER_DAY)) for i in range(s3) ])
        #print(e)
        rs.append(e)
    #print(rs)
    s = np.concatenate(rs)
    #print(s)
    g_arrivals = pd.DataFrame(data=s, columns=['value'])
    _demands = [0 for i in range(0, N_PERIODS+1)]
    for t in s:
        _demands[t] = _demands[t] +1
    demands = pd.DataFrame(data= [(t, _demands[t]) for t in range(N_PERIODS)], columns = ['period', 'demand'])
    return demands


The number of scenarios you want to generate and solve:

In [4]:
N_SCENARIOS = 5


When copying the scenario, copy the input data, the model and the solution if any.

Then attach new randomly generated data and solve.

Grab the solution to perform some multi scenario reporting in this notebook.

In [5]:
all_kpis = pd.DataFrame()

for i in range(1, N_SCENARIOS+1):
    sc_name = "Copy %02d" % (i)
    print(sc_name)
    copy = decision.get_scenario(name=sc_name)
    if (copy != None):
        print(" Deleting old...")
        decision.delete_container(copy)
    print(" Copying from original scenario...")    
    copy = scenario.copy(sc_name)
    print(" Generating new demand...")
    df_demands = random_demand(200)
    copy.add_table_data("demands", df_demands, category='input')
    print(" Solving...")
    copy.solve()
    print(" Grabbing solution kpis...")
    kpis = copy.get_table_data('kpis')
    kpis['scenario'] = sc_name
    mk = [[ kpis.iloc[0]['VALUE'],  "%02d" % (kpis.iloc[1]['VALUE']), sc_name, "%02d" % (kpis.iloc[2]['VALUE'])]]
    my_kpis = pd.DataFrame(data=mk, columns=['cost','fix','scenario','temp'])
    copy.add_table_data('my_kpis', data=my_kpis, category='output')
    all_kpis = all_kpis.append(kpis)
    
print("Done!")

Copy 01
 Deleting old...
 Copying from original scenario...
 Generating new demand...
 Solving...
[2019-10-17T14:25:55Z, INFO] Reduced MIP has 0 binaries, 304 generals, 0 SOSs, and 0 indicators.
[2019-10-17T14:25:55Z, INFO] Presolve time = 0.00 sec. (1.52 ticks)
[2019-10-17T14:25:55Z, INFO] MIP emphasis: balance optimality and feasibility.
[2019-10-17T14:25:55Z, INFO] MIP search method: dynamic search.
[2019-10-17T14:25:55Z, INFO] Parallel mode: none, using 1 thread.
[2019-10-17T14:25:55Z, INFO] Root relaxation solution time = 0.00 sec. (0.89 ticks)
[2019-10-17T14:25:55Z, INFO] 
[2019-10-17T14:25:55Z, INFO]         Nodes                                         Cuts/
[2019-10-17T14:25:55Z, INFO]    Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap
[2019-10-17T14:25:55Z, INFO] *     0+    0                         6460.0000     1000.0000            84.52%
[2019-10-17T14:25:55Z, INFO] *     0     0      integral     0     5740.0000     5740.0000       79    0.00%

### Reporting

Display multi scenario comparison report.

In [6]:
import brunel 

total_cost = all_kpis[all_kpis.NAME=='Total Cost']

%brunel data('total_cost') x(scenario) y(value:linear) color(scenario) tooltip(#all) bar :: width=1500, height=300

<IPython.core.display.Javascript object>

Copyright &copy; 2017-2022 IBM. This notebook and its source code are released under the terms of the MIT License.